In [15]:
import os
import pandas as pd
import json

In [16]:
langs = ['it', 'en', 'es']
corrections = {}
for source_lang in langs:
    for target_lang in langs:
        if source_lang != target_lang:
            #print(f"Processing translation from {source_lang} to {target_lang}")
            folder_path = f"final_evaluated_output/{source_lang}_to_{target_lang}/"
            for file_name in os.listdir(folder_path):
                if file_name.startswith(f"{source_lang}_to_{target_lang}_") and file_name.endswith(".xlsx"):
                    file_path = os.path.join(folder_path, file_name)
                    df = pd.read_excel(file_path)
                    #print(f"Loaded {file_name} with {len(df)} rows")

                    for index, row in df.iterrows():
                        entry_id = row['unique id']
                        # find if isset row 'Corrected source'
                        source_correction = row['Corrected source'] if 'Corrected source' in row else None
                        target_correction = row['Corrected translation']

                        if pd.notna(target_correction) or pd.notna(source_correction):
                            if not entry_id in corrections:
                                corrections[entry_id] = {
                                    'en': None,
                                    'it': None,
                                    'es': None
                                }
                            
                        if pd.notna(target_correction):
                            corrections[entry_id][target_lang] = target_correction

                        if pd.notna(source_correction):
                            corrections[entry_id][source_lang] = source_correction

corrections
with open("corrections.json", 'w', encoding='utf-8') as f:
    json.dump(corrections, f, ensure_ascii=False, indent=4)

In [17]:
corpus_path = "../output/run_20251120_100000/unified_corpus.json"
with open(corpus_path, 'r', encoding='utf-8') as f:
    corpus = json.load(f)

print(f"Number of entries in the corpus: {len(corpus)}")

Number of entries in the corpus: 616


In [18]:
for entry in corpus:
    entry_id = entry['unique_id']
    if entry_id in corrections:
        for lang in langs:
            if corrections[entry_id][lang] is not None:
                entry[f"annotation_{lang}"] = corrections[entry_id][lang]
                entry[f"annotation_{lang}_is_gold"] = True

with open("corrected_unified_corpus.json", 'w', encoding='utf-8') as f:
    json.dump(corpus, f, ensure_ascii=False, indent=4)
